pandas是贯穿后续部分的主要工具。它所包含的数据结构和数据处理工具的设计使得在Python中进行数据清洗和分析非常快捷。pandas经常是和其他数值计算工具，比如NumPy和SciPy，以及数据可视化工具比如matplotlib一起使用的。pandas支持大部分NumPy语言风格的数组计算，尤其是数组函数以及没有for循环的各种数据处理。

尽管pandas采用了很多NumPy的代码风格，但最大的不同在于pandas是用来处理表格型或异质型数据的。而NumPy则相反，它更适合处理同质型的数值类数组数据。

2010年成为了开源项目，pandas已经逐渐成熟，成为一个在真实世界中广泛应用的大型类库。pandas的开发者社区已经有超过800个代码贡献者，他们帮助构建了项目，并将pandas应用到日常中去解决他们的数据难题。

贯穿本书的后续章节，会使用下面的便捷方式导入pandas：

In [1]:
import pandas as pd
import numpy as np

因此，无论何时，只要你在代码中看到`pd.`，表示对pandas的引用。还可以方便地从本地命名空间中导入Series和DataFrame，它们是常用的类：

In [2]:
from pandas import Series, DataFrame

## 5.1　pandas数据结构介绍
为了入门pandas，需要熟悉两个常用的工具数据结构：Series和DataFrame。尽管他们并不能解决所有的问题，但它们为大多数应用提供了一个有效、易用的基础。

### 5.1.1　Series
Series是一种一维的数组型对象，它包含了一个值序列（与NumPy中的类型相似），并且包含了数据标签，称为索引（index）。最简单的序列可以仅仅由一个数组形成：

In [3]:
obj = pd.Series([4, 7, -5, 3])

print(obj)

0    4
1    7
2   -5
3    3
dtype: int64


交互式环境中Series的字符串表示，索引在左边，值在右边。由于不为数据指定索引，默认生成的索引是从0到N-1（N是数据的长度）。可以通过values属性和index属性分别获得Series对象的值和索引：

In [4]:
obj.values

array([ 4,  7, -5,  3])

In [5]:
obj.index  # 与 range(4)类似

RangeIndex(start=0, stop=4, step=1)

通常需要创建一个索引序列，用标签标识每个数据点：

In [6]:
obj2 = pd.Series([4, 7, -5, 3], index=['d', 'b', 'a', 'c'])

obj2

d    4
b    7
a   -5
c    3
dtype: int64

In [7]:
print(obj2.index, obj2.values)

Index(['d', 'b', 'a', 'c'], dtype='object') [ 4  7 -5  3]


与NumPy的数组相比，可以在从数据中选择数据的时候使用标签来进行索引：

In [8]:
obj2['a']

obj2['d'] = 6

obj2[['c', 'a', 'd']]

c    3
a   -5
d    6
dtype: int64

上面的例子中，`['c'，'a'，'d']`包含的不是数字而是字符串，作为索引列表。

使用NumPy的函数或NumPy风格的操作，比如使用布尔值数组进行过滤，与标量相乘，或是应用数学函数，这些操作将保存索引值连接：

In [9]:
print(obj2[obj2 > 0])
print('-'*50)
print(obj2 * 2)
print('-'*50)
print(np.exp(obj2))

d    6
b    7
c    3
dtype: int64
--------------------------------------------------
d    12
b    14
a   -10
c     6
dtype: int64
--------------------------------------------------
d     403.428793
b    1096.633158
a       0.006738
c      20.085537
dtype: float64


从另一个角度考虑Series，可以认为它是一个`长度固定且有序的字典`，因为它将索引值和数据值按位置配对。在可能会使用字典的上下文中，也可以使用Series：

In [10]:
print('b' in obj2)

print('e' in obj2)

True
False


如果已经有数据包含在Python字典中，可以使用字典生成一个Series：

In [11]:
sdata = {'Ohio': '35000l', 'Texas': 71000, 'Oregon': 16000, 'Utah': 5000}
obj3 = pd.Series(sdata)
print(obj3)

sdata = {'Ohio': 35000, 'Texas': 71000, 'Oregon': 16000, 'Utah': 5000}
obj3 = pd.Series(sdata)
print(obj3)

Ohio      35000l
Texas      71000
Oregon     16000
Utah        5000
dtype: object
Ohio      35000
Texas     71000
Oregon    16000
Utah       5000
dtype: int64


当把字典传递给Series构造函数时，产生的Series的索引将是排序好的字典键。可以将字典键按照所想要的顺序传递给构造函数，从而使生成的Series的索引顺序符合你的预期：

In [12]:
states = ['California', 'Ohio', 'Oregon', 'Texas']
obj4 = pd.Series(sdata, index=states, dtype=np.int64)
print(obj4)
# NaN是不能转为int类型的
# obj4.astype(np.int64)

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64


上面的例子中，sdata中的三个值被放置在正确的位置，但是因为'California'没有出现在sdata的键中，它对应的值是NaN（not a number），这是pandas中标记缺失值或NA值的方式。因为'Utah'并不在states中，它被排除在结果对象外。

后面将持续使用术语“缺失”或“NA”来表示缺失数据。pandas中使用isnull和notnull函数来检查缺失数据：

In [13]:
print(pd.isnull(obj4))
print('-'*50)
print(pd.notnull(obj4))

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool
--------------------------------------------------
California    False
Ohio           True
Oregon         True
Texas          True
dtype: bool


isnull和notnull也是Series的实例方法：

In [14]:
obj4.isnull()

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

对于很多应用来说，在数学操作中自动对齐索引是Series的一个非常有用的特性：

In [15]:
print(obj3)
print('-'*50)
print(obj4)
print('-'*50)
print(obj3 + obj4)

Ohio      35000
Texas     71000
Oregon    16000
Utah       5000
dtype: int64
--------------------------------------------------
California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64
--------------------------------------------------
California         NaN
Ohio           70000.0
Oregon         32000.0
Texas         142000.0
Utah               NaN
dtype: float64


数据对齐特性将在后续深入讨论。如果有数据库经验，可以认为这个特性与数据库的join操作是非常相似的。

Series对象自身和其索引都有name属性，这个特性与pandas其他重要功能集成在一起：

In [16]:
obj4.name = 'population'
obj4.index.name = 'state'
print(obj4)

state
California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
Name: population, dtype: float64


Series的索引可以通过按位置赋值的方式进行改变：

In [17]:
print(obj)
print('-'*50)
obj.index = ['Bob', 'Steve', 'Jeff', 'Ryan']
print(obj, type(obj))

0    4
1    7
2   -5
3    3
dtype: int64
--------------------------------------------------
Bob      4
Steve    7
Jeff    -5
Ryan     3
dtype: int64 <class 'pandas.core.series.Series'>


### 5.1.2　DataFrame
DataFrame表示的是矩阵的数据表，它包含已排序的列集合，每一列可以是不同的值类型（数值、字符串、布尔值等）。DataFrame既有行索引也有列索引，它可以被视为一个共享相同索引的Series的字典。在DataFrame中，数据被存储为一个以上的二维块，而不是列表、字典或其他一维数组的集合。

尽管DataFrame是二维的，但可以利用分层索引在DataFrame中展现更高维度的数据。分层索引是pandas中一种更为高级的数据处理特性。

有多种方式可以构建DataFrame，其中最常用的方式是利用包含等长度列表或NumPy数组的字典来形成DataFrame：

In [18]:
data = {'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada', 'Nevada'],
        'year': [2000, 2001, 2002, 2001, 2002, 2003],
        'pop': [1.5, 1.7, 3.6, 2.4, 2.9, 3.2]}
frame = pd.DataFrame(data)

产生的DataFrame会自动为Sereies分配索引，并且列会按照排序的顺序排列：

In [19]:
frame

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9
5,Nevada,2003,3.2


如果使用的是Jupyter notebook，pandas的DataFrame对象将会展示为一个对浏览器更为友好的HTML表格。

对于大型DataFrame，head方法将会只选出头部的五行：

In [20]:
frame.head()

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9


如果指定了列的顺序，DataFrame的列将会按照指定顺序排列：

In [21]:
pd.DataFrame(data, columns=['year', 'state', 'pop'])

,year,state,pop
0,2000,Ohio,1.5
1,2001,Ohio,1.7
2,2002,Ohio,3.6
3,2001,Nevada,2.4
4,2002,Nevada,2.9
5,2003,Nevada,3.2


如果传的列不包含在字典中，将会在结果中出现缺失值：

In [22]:
frame2 = pd.DataFrame(data, columns=['year', 'state', 'pop', 'debt'],
                      index=['one', 'two', 'three', 'four','five', 'six'])

print(frame2, type(frame2))
print('-'*50)
print(frame2.columns, type(frame2.columns))

       year   state  pop debt
one    2000    Ohio  1.5  NaN
two    2001    Ohio  1.7  NaN
three  2002    Ohio  3.6  NaN
four   2001  Nevada  2.4  NaN
five   2002  Nevada  2.9  NaN
six    2003  Nevada  3.2  NaN <class 'pandas.core.frame.DataFrame'>
--------------------------------------------------
Index(['year', 'state', 'pop', 'debt'], dtype='object') <class 'pandas.core.indexes.base.Index'>


DataFrame中的一列，可以按字典型标记或属性那样检索为Series：

In [23]:
print(frame2['state'], type(frame2['state']))
print('-'*50)
print(frame2.year, type(frame2.pop))

one        Ohio
two        Ohio
three      Ohio
four     Nevada
five     Nevada
six      Nevada
Name: state, dtype: object <class 'pandas.core.series.Series'>
--------------------------------------------------
one      2000
two      2001
three    2002
four     2001
five     2002
six      2003
Name: year, dtype: int64 <class 'method'>


在IPython中，属性型连接（比如`frame2.year`）和列名的tab补全是非常方便的。

`frame2[colunm]`对于任意列名均有效，但是`frame2.column`只在列名是有效的Python变量名时有效。

请注意，返回的Series与原DataFrame有相同的索引，且Series的name属性也会被合理地设置。

行也可以通过位置或特殊属性`loc`进行选取（后续详细介绍）：

In [24]:
frame2.loc['three'], type(frame2.loc['three'])

(year     2002
 state    Ohio
 pop       3.6
 debt      NaN
 Name: three, dtype: object, pandas.core.series.Series)

列的引用是可以修改的。例如，空的'debt'列可以赋值为标量值或值数组：

In [25]:
frame2['debt'] = 16.5

print(frame2)
print('-'*50)
frame2['debt'] = np.arange(6.0)
print(frame2)
print('-'*50)
frame2['debt'] = range(-6,0)
print(frame2)

       year   state  pop  debt
one    2000    Ohio  1.5  16.5
two    2001    Ohio  1.7  16.5
three  2002    Ohio  3.6  16.5
four   2001  Nevada  2.4  16.5
five   2002  Nevada  2.9  16.5
six    2003  Nevada  3.2  16.5
--------------------------------------------------
       year   state  pop  debt
one    2000    Ohio  1.5   0.0
two    2001    Ohio  1.7   1.0
three  2002    Ohio  3.6   2.0
four   2001  Nevada  2.4   3.0
five   2002  Nevada  2.9   4.0
six    2003  Nevada  3.2   5.0
--------------------------------------------------
       year   state  pop  debt
one    2000    Ohio  1.5    -6
two    2001    Ohio  1.7    -5
three  2002    Ohio  3.6    -4
four   2001  Nevada  2.4    -3
five   2002  Nevada  2.9    -2
six    2003  Nevada  3.2    -1


当将列表或数组赋值给一个列时，值的长度必须和DataFrame的长度相匹配。如果将Series赋值给一列时，Series的索引将会按照DataFrame的索引重新排列，并在空缺的地方填充缺失值：

In [26]:
val = pd.Series([-1.2, -1.5, -1.7], index=['two', 'four', 'five'])

frame2['debt'] = val
print(frame2)

       year   state  pop  debt
one    2000    Ohio  1.5   NaN
two    2001    Ohio  1.7  -1.2
three  2002    Ohio  3.6   NaN
four   2001  Nevada  2.4  -1.5
five   2002  Nevada  2.9  -1.7
six    2003  Nevada  3.2   NaN


如果被赋值的列并不存在，则会生成一个新的列。`del`关键字可以像在字典中那样对DataFrame删除列。

在`del`的例子中，首先增加一列，这一列是布尔值，判断条件是state列是否为'Ohio'：

In [27]:
frame2['eastern'] = frame2.state == 'Ohio'

frame2

,year,state,pop,debt,eastern
one,2000,Ohio,1.5,NaN,True
two,2001,Ohio,1.7,-1.2,True
three,2002,Ohio,3.6,NaN,True
four,2001,Nevada,2.4,-1.5,False
five,2002,Nevada,2.9,-1.7,False
six,2003,Nevada,3.2,NaN,False


> frame2.eastern的语法无法创建新的列。

`del`方法可以用于移除之前新建的列：

In [28]:
del frame2['eastern']

print(frame2.columns)
print(frame2)

Index(['year', 'state', 'pop', 'debt'], dtype='object')
       year   state  pop  debt
one    2000    Ohio  1.5   NaN
two    2001    Ohio  1.7  -1.2
three  2002    Ohio  3.6   NaN
four   2001  Nevada  2.4  -1.5
five   2002  Nevada  2.9  -1.7
six    2003  Nevada  3.2   NaN


> 从DataFrame中选取的列是数据的`视图`，而不是拷贝。因此，对Series的修改会映射到DataFrame中。如果需要复制，则应当显式地使用Series的`copy`方法。

另一种常用的数据形式是包含字典的嵌套字典

In [29]:
pop = {'Nevada': {2001: 2.4, 2002: 2.9},
       'Ohio': {2000: 1.5, 2001: 1.7, 2002: 3.6}}

如果嵌套字典被赋值给DataFrame，pandas会将**字典的键作为列，将内部字典的键作为行索引**：

In [30]:
frame3 = pd.DataFrame(pop)

frame3

,Nevada,Ohio
2000,NaN,1.5
2001,2.4,1.7
2002,2.9,3.6


可以将使用类似NumPy的语法对DataFrame进行转置操作（调换行和列）：

In [31]:
frame3.T

,2000,2001,2002
Nevada,NaN,2.4,2.9
Ohio,1.5,1.7,3.6


内部字典的键被联合、排序后形成了结果的索引。如果已经显式指明索引的话，内部字典的键将不会被排序：

In [32]:
pd.DataFrame(pop, index=[2001, 2002, 2003])

,Nevada,Ohio
2001,2.4,1.7
2002,2.9,3.6
2003,NaN,NaN


包含Series的字典也可以用于构造DataFrame：

In [33]:
print(frame3)
print('-'*50)
pdata = {'Ohio': frame3['Ohio'][:-1],
         'Nevada': frame3['Nevada'][:2]}
print(pd.DataFrame(pdata))

      Nevada  Ohio
2000     NaN   1.5
2001     2.4   1.7
2002     2.9   3.6
--------------------------------------------------
      Ohio  Nevada
2000   1.5     NaN
2001   1.7     2.4


可以向DataFrame构造函数传递的对象列表见下表。

|          类型           |                                       注释                                        |
| ---------------------- | -------------------------------------------------------------------------------- |
| 2D ndarray             | 数据的矩阵,行和列的标签是可选参数                                                     |
| 数组、列表和元组构成的字典 | 每个序列成为 Data frame的一列,所有的序列必须长度相等                                   |
| NumPy结构化/记录化数组   | 与数组构成的字典一致                                                                |
| Series构成的字典         | 每个值成为一列,每个 Series的索引联合起来形成结果的行索引,也可以显式地传递索引              |
| 字典构成的字典           | 每一个内部字典成为一列,键联合起来形成结果的行索引                                       |
| 字典或 Series构成的列表  | 列表中的一个元素形成 Data Frame的一行,字典键或 Series索引联合起来形成 Data frame的列标签 |
| 列表或元组构成的列表      | 与2D ndarray的情况一致                                                             |
| 其他 DataFrame          | 如果不显示传递索引,则会使用原 Data frame的索引                                        |
| NumPy MaskedArray      | 与2D ndarray的情况类似,但隐蔽值会在结果 Data Frame中成为NA/缺失值                      |

如果DataFrame的索引和列拥有`name`属性，则这些`name`属性也会被显示：

In [34]:
frame3.index.name = 'year'; frame3.columns.name = 'state'
frame3

state,Nevada,Ohio
year,,
2000,NaN,1.5
2001,2.4,1.7
2002,2.9,3.6


和Series类似，DataFrame的values属性会将包含在DataFrame中的数据以二维ndarray的形式返回：

In [35]:
frame3.values, frame3.values.dtype

(array([[nan, 1.5],
        [2.4, 1.7],
        [2.9, 3.6]]), dtype('float64'))

如果DataFrame的列是不同的dtypes，则values的dtype会自动选择适合所有列的类型：

In [36]:
frame2.values

array([[2000, 'Ohio', 1.5, nan],
       [2001, 'Ohio', 1.7, -1.2],
       [2002, 'Ohio', 3.6, nan],
       [2001, 'Nevada', 2.4, -1.5],
       [2002, 'Nevada', 2.9, -1.7],
       [2003, 'Nevada', 3.2, nan]], dtype=object)

### 5.1.3　索引对象
pandas中的索引对象是用于存储轴标签和其他元数据的（例如轴名称或标签）。在构造Series或DataFrame时，所使用的任意数组或标签序列都可以在内部转换为索引对象：

In [37]:
obj = pd.Series(range(3), index=['a', 'b', 'c'])
index = obj.index

print(index)
print(index[1:])

Index(['a', 'b', 'c'], dtype='object')
Index(['b', 'c'], dtype='object')


索引对象是不可变的，因此用户是无法修改索引对象的：

In [38]:
index[1] = 'd'  # TypeError

TypeError: Index does not support mutable operations

不变性使得在多种数据结构中分享索引对象更为安全：

In [39]:
labels = pd.Index(np.arange(3))
print(labels)

print("-"*50)
obj2 = pd.Series([1.5, -2.5, 0], index=labels)
print(obj2)

print("-"*50)
print(obj2.index is labels)

Int64Index([0, 1, 2], dtype='int64')
--------------------------------------------------
0    1.5
1   -2.5
2    0.0
dtype: float64
--------------------------------------------------
True


 一些用户并不经常利用索引对象提供的功能，但是因为一些操作会产生包含索引化数据的结果，理解索引如何工作还是很重要的。

除了类似数组，索引对象也像一个固定大小的集合：

In [40]:
print(frame3)
print("-"*50)
print(frame3.columns)
print("-"*50)
print('Ohio' in frame3.columns)
print("-"*50)
print(2002 in frame3.index)
print(2003 in frame3.index)

state  Nevada  Ohio
year               
2000      NaN   1.5
2001      2.4   1.7
2002      2.9   3.6
--------------------------------------------------
Index(['Nevada', 'Ohio'], dtype='object', name='state')
--------------------------------------------------
True
--------------------------------------------------
True
False


> 与Python集合不同，pandas索引对象可以包含重复标签：

In [41]:
dup_labels = pd.Index(['foo', 'foo', 'bar', 'bar'])

print(dup_labels)
print(dup_labels.unique())
print('|'.join(dup_labels.unique()))

Index(['foo', 'foo', 'bar', 'bar'], dtype='object')
Index(['foo', 'bar'], dtype='object')
foo|bar


根据重复标签进行筛选，会选取所有重复标签对应的数据。

每个索引都有一些集合逻辑的方法和属性，这些方法和属性解决了关于它所包含的数据的其他常见问题。下表总结了这些方法和属性中常用的一部分。


| 方法          | 描述                                      |
| -            | -                                        |
| append       | 将额外的索引对象粘贴到原索引后,产生一个新的索引 |
| difference   | 计算两个索引的差集                          |
| intersection | 计算两个索引的交集                          |
| union        | 计算两个索引的并集                          |
| isin         | 计算表示每一个值是否在传值容器中的布尔数组     |
| delete       | 将位置i的元素删除,并产生新的索引              |
| drop         | 根据传参删除指定索引值,并产生新的索引          |
| insert       | 在位置i插入元素,并产生新的索引               |
| is monotonic | 如果索引序列递增则返回True                   |
| Is unique    | 如果索引序列唯一则返回True                   |
| unique       | 计算索引的唯一值序列                        |


## 5.2　基本功能
接下来将会指引了解与Series或DataFrame中数据交互的基础机制。后续的内容中会更为深入地讲解使用pandas进行数据分析和操作的主题。

### 5.2.1　重建索引
reindex是pandas对象的重要方法，该方法用于创建一个符合新索引的新对象。考虑下面的例子：

In [42]:
obj = pd.Series([4.5, 7.2, -5.3, 3.6], index=['d', 'b', 'a', 'c'])

obj

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

Series调用reindex方法时，会将数据按照新的索引进行排列，如果某个索引值之前并不存在，则会引入缺失值：

In [43]:
obj2 = obj.reindex(['a', 'b', 'c', 'd', 'e'])

obj2

a   -5.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

对于顺序数据，比如时间序列，在重建索引时可能会需要进行插值或填值。`method`可选参数允许使用诸如`ffill`等方法在重建索引时插值，`ffill`方法会将值前向填充：

In [44]:
obj3 = pd.Series(['blue', 'purple', 'yellow'], index=[0, 2, 4])
obj3

obj3.reindex(range(6), method='ffill')

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object

在DataFrame中，reindex可以改变行索引、列索引，也可以同时改变二者。当仅传入一个序列时，结果中的行会重建索引：

In [45]:
frame = pd.DataFrame(np.arange(9).reshape((3, 3)),
                     index=['a', 'c', 'd'],
                     columns=['Ohio', 'Texas', 'California'])

frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [46]:
frame2 = frame.reindex(['a', 'b', 'c', 'd'])

frame2

,Ohio,Texas,California
a,0.0,1.0,2.0
b,NaN,NaN,NaN
c,3.0,4.0,5.0
d,6.0,7.0,8.0


列可以使用columns关键字重建索引：

In [47]:
states = ['Texas', 'Utah', 'California']
frame.reindex(columns=states)

,Texas,Utah,California
a,1,NaN,2
c,4,NaN,5
d,7,NaN,8


下表是reindex方法的参数列表。

|    参数     |                                         描述                                         |
| ---------- | ------------------------------------------------------------------------------------ |
| index      | 新建作为索引的序列,<br>可以是索引实例或任意其他序列型Python数据结构索引使用时无须复制             |
| method     | 插值方式;'ffill'为前向填充,而'bfill'是后向填充                                          |
| fill_value | 通过重新索引引入缺失数据时使用的替代值                                                    |
| limit      | 当前向或后向填充时,所需填充的最大尺寸间隙(以元素数量)                                       |
| tolerance  | 当前向或后向填充时,<br>所需填充的不精确匹配下的最大尺寸间隙(以绝对数字距离)                       |
| level      | 匹配 MultiIndex级别的简单索引;否则选择子集                                               |
| copy       | 如果为True,即使新索引等于旧索引,也总是复制底层数据;<br>如果是 False,则在索引相同时不要复制数据 |

更深入地探索时，可以使用loc进行更为简洁的标签索引，许多用户更倾向于使用这种方式：

In [48]:
# 不增加数据内容的，不会产生报错
print(frame.loc[['a','c','d'],['Ohio','Texas']])
print("-"*50)
# 会有后期功能不可用的警告(提示使用reindex)
print(frame.loc[['a', 'b', 'c', 'd'], states])

   Ohio  Texas
a     0      1
c     3      4
d     6      7
--------------------------------------------------
   Texas  Utah  California
a    1.0   NaN         2.0
b    NaN   NaN         NaN
c    4.0   NaN         5.0
d    7.0   NaN         8.0


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


### 5.2.2　轴向上删除条目
如果已经拥有索引数组或不含条目的列表，在轴向上删除一个或更多的条目就非常容易，但这样需要一些数据操作和集合逻辑，`drop`方法会返回一个含有指示值或轴向上删除值的新对象：

In [49]:
obj = pd.Series(np.arange(5.), index=['a', 'b', 'c', 'd', 'e'])

obj

a    0.0
b    1.0
c    2.0
d    3.0
e    4.0
dtype: float64

In [50]:
new_obj = obj.drop('c')
print(new_obj)
print("-"*50)
print(obj.drop(['d', 'c']))

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64
--------------------------------------------------
a    0.0
b    1.0
e    4.0
dtype: float64


在DataFrame中，索引值可以从轴向上删除。为了表明这个特性，首先创建一个示例DataFrame：

In [51]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])

data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


在调用drop时使用标签序列会根据行标签删除值（轴0）：

In [52]:
data.drop(['Colorado', 'Ohio'])

,one,two,three,four
Utah,8,9,10,11
New York,12,13,14,15


可以通过传递`axis=1`或`axis='columns'`来从列中删除值：

In [53]:
print(data.drop('two', axis=1))
print("-"*50)
print(data.drop(['one', 'four'], axis='columns'))

          one  three  four
Ohio        0      2     3
Colorado    4      6     7
Utah        8     10    11
New York   12     14    15
--------------------------------------------------
          two  three
Ohio        1      2
Colorado    5      6
Utah        9     10
New York   13     14


很多函数，例如`drop`，会修改Series或DataFrame的尺寸或形状，这些方法直接操作原对象而不返回新对象：

In [54]:
obj.drop('c', inplace=True)
obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

> 请注意`inplace`属性，它会清除被删除的数据。

### 5.2.3　索引、选择与过滤
Series的索引（obj[...]）与NumPy数组索引的功能类似，只不过Series的索引值可以不仅仅是整数。相关示例如下：

In [55]:
obj = pd.Series(np.arange(4.), index=['a', 'b', 'c', 'd'])

obj

a    0.0
b    1.0
c    2.0
d    3.0
dtype: float64

In [56]:
print(obj['b'])
print(obj[1])
print(obj[2:4])
print(obj[['b', 'a', 'd']])
print(obj[[1, 3]])
print(obj[obj < 2])

1.0
1.0
c    2.0
d    3.0
dtype: float64
b    1.0
a    0.0
d    3.0
dtype: float64
b    1.0
d    3.0
dtype: float64
a    0.0
b    1.0
dtype: float64


普通的Python切片中是不包含尾部的，Series的切片与之不同：

In [57]:
print(obj['b':'c'])

b    1.0
c    2.0
dtype: float64


使用这些方法设值时会修改Series相应的部分：

In [58]:
obj['b':'c'] = 5

print(obj)

a    0.0
b    5.0
c    5.0
d    3.0
dtype: float64


使用单个值或序列，可以从DataFrame中索引出一个或多个列：

In [59]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])

print(data)
print("-"*50)
print(data['two'])
print("-"*50)
print(data[['three', 'one']])
print("-"*50)
print(data[['three', 'one']][1:3])

          one  two  three  four
Ohio        0    1      2     3
Colorado    4    5      6     7
Utah        8    9     10    11
New York   12   13     14    15
--------------------------------------------------
Ohio         1
Colorado     5
Utah         9
New York    13
Name: two, dtype: int64
--------------------------------------------------
          three  one
Ohio          2    0
Colorado      6    4
Utah         10    8
New York     14   12
--------------------------------------------------
          three  one
Colorado      6    4
Utah         10    8


这种索引方式也有特殊案例。首先，可以根据一个布尔值数组切片或选择数据：

In [60]:
print(data[:2])
print("-"*50)
print(data[data['three'] > 5])
data['three'] > 5

          one  two  three  four
Ohio        0    1      2     3
Colorado    4    5      6     7
--------------------------------------------------
          one  two  three  four
Colorado    4    5      6     7
Utah        8    9     10    11
New York   12   13     14    15


Ohio        False
Colorado     True
Utah         True
New York     True
Name: three, dtype: bool

行选择语法`data[：2]`非常方便。传递单个元素或一个列表到[]符号中可以选择列。

另一个用例是使用布尔值DataFrame进行索引，布尔值DataFrame可以是对标量值进行比较产生的：

In [61]:
data < 5

,one,two,three,four
Ohio,True,True,True,True
Colorado,True,False,False,False
Utah,False,False,False,False
New York,False,False,False,False


In [62]:
data[data < 5] = 0

data

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


在这个特殊例子中，这种索引方式使得DataFrame在语法上更像是NumPy二维数组。

#### 5.2.3.1　使用loc和iloc选择数据

针对DataFrame在行上的标签索引，将介绍特殊的索引符号loc和iloc。他们允许使用轴标签（`loc`）或整数标签（`iloc`）以NumPy风格的语法从DataFrame中选出数组的行和列的子集。

通过标签选出单行多列的数据作为基础示例：

In [63]:
print(data.loc[['Colorado','Utah'], ['two', 'three']])

          two  three
Colorado    5      6
Utah        9     10


然后使用整数标签`iloc`进行类似的数据选择：

In [64]:
data.iloc[2, [3, 0, 1]]

four    11
one      8
two      9
Name: Utah, dtype: int64

In [65]:
data.iloc[[1, 2], [3, 0, 1]]

,four,one,two
Colorado,7,0,5
Utah,11,8,9


除了单个标签或标签列表之外，索引功能还可以用于切片：

In [66]:
data.loc[:'Utah', ['two']]

,two
Ohio,0
Colorado,5
Utah,9


In [67]:
data.iloc[:, :3][data.three > 5]

,one,two,three
Colorado,0,5,6
Utah,8,9,10
New York,12,13,14


因此，有多种方式可以选择、重排pandas对象中的数据。下表提供了DataFrame选择数据的一些方法。之后的章节会介绍处理分层索引的可选参数。

|             类型              |                                                      描述                                                       |
| ----------------------------- | -------------------------------------------------------------------------------------------------------------- |
| df\[val\]                     | 从 Dataframe中选择单列或列序列;<br>特殊情况的便利:布尔数组(过滤行),切片(切片行)或布尔值 Data frame(根据某些标准设置的值) |
| df.loc\[val\]                 | 根据标签选择 DataFrame的单行或多行                                                                                |
| df.loc\[:, val\]              | 根据标签选择单列或多列                                                                                            |
| df.loc\[val1, val2\]          | 同时选择行和列中的一部分                                                                                          |
| df.iloc\[where\]              | 根据整数位置选择单行或多行                                                                                        |
| df.loc\[:, where\]            | 根据整数位置选择单列或多列                                                                                        |
| df.iloc\[where\_i, where\_j\] | 根据整数位置选择行和列                                                                                            |
| df.at\[label\_i, label\_j\]   | 根据行、列标签选择单个标量值                                                                                       |
| df.at\[i, j\]                 | 根据行、列整数位置选择单个标量值                                                                                    |
| reindex方法                   | 通过标签选择行或列                                                                                               |
| get\_value, set\_value方法    | 根据行和列的标签设置单个值                                                                                        |
  

> 在早期设计pandas时，我认为使用`frame[:, col]`来选择一列太过冗余（也容易出错），这是因为列选择是最为常用的操作。我做了设计权衡，将所有的神奇索引行为（包括标签和整数）都放到`ix`操作符中。事实上，这导致了整数轴标签数据中的许多边缘情况，所以pandas团队决定创建loc和iloc运算符分别用于严格处理基于标签和基于整数的索引。

> `ix`索引依然存在，但已经弃用。并不推荐使用它。

### 5.2.4　整数索引
在pandas对象使上用整数索引对新用户来说经常会产生歧义，这是因为它和在列表、元组等Python内建数据结构上进行索引有些许不同。例如，下面的代码会产生错误：

In [68]:
ser = pd.Series(np.arange(3.))
print(ser)
ser[-1]

0    0.0
1    1.0
2    2.0
dtype: float64


KeyError: -1

在上面的例子中，pandas可以“回退”到整数索引，但是这样的方式难免会引起一些微小的错误。假设有一个索引，它包含了0、1、2，但是推断用户所需要的索引方式（标签索引或位置索引）是很难的：

In [69]:
ser

0    0.0
1    1.0
2    2.0
dtype: float64

另一方面，对于非整数索引，则不会有潜在的歧义：

In [70]:
ser2 = pd.Series(np.arange(3.), index=['a', 'b', 'c'])

print(ser2[-1])

2.0


为了保持一致性，如果有一个包含整数的轴索引，数据选择时请始终使用标签索引。为了更精确地处理，可以使用`loc`（用于标签）或`iloc`（用于整数）：

In [71]:
print(ser[:1])
print("-"*50)
print(ser.loc[:1])
print("-"*50)
# loc使用-1无法取得正确结果
print(ser.loc[:-1])
print("-"*50)
print(ser.iloc[:-1])

0    0.0
dtype: float64
--------------------------------------------------
0    0.0
1    1.0
dtype: float64
--------------------------------------------------
Series([], dtype: float64)
--------------------------------------------------
0    0.0
1    1.0
dtype: float64


### 5.2.5　算术和数据对齐
不同索引的对象之间的算术行为是pandas提供给一些应用的一项重要特性。当将对象相加时，如果存在某个索引对不相同，则返回结果的索引将是索引对的并集。对数据库用户来说，这个特性类似于索引标签的自动`外连接`（`outer join`）。看下面的示例：

In [72]:
s1 = pd.Series([7.3, -2.5, 3.4, 1.5], index=['a', 'c', 'd', 'e'])

s2 = pd.Series([-2.1, 3.6, -1.5, 4, 3.1],
               index=['a', 'c', 'e', 'f', 'g'])

print(s1)
print("-"*50)
print(s2)

a    7.3
c   -2.5
d    3.4
e    1.5
dtype: float64
--------------------------------------------------
a   -2.1
c    3.6
e   -1.5
f    4.0
g    3.1
dtype: float64


将这些对象相加会产生：

In [73]:
s1 + s2

a    5.2
c    1.1
d    NaN
e    0.0
f    NaN
g    NaN
dtype: float64

没有交叠的标签位置上，内部数据对齐会产生缺失值。缺失值会在后续的算术操作上产生影响。在DataFrame的示例中，行和列上都会执行对齐：

In [74]:
df1 = pd.DataFrame(np.arange(9.).reshape((3, 3)), columns=list('bcd'),
                   index=['Ohio', 'Texas', 'Colorado'])

df2 = pd.DataFrame(np.arange(12.).reshape((4, 3)), columns=list('bde'),
                   index=['Utah', 'Ohio', 'Texas', 'Oregon'])

print(df1)
print("-"*50)
print(df2)

            b    c    d
Ohio      0.0  1.0  2.0
Texas     3.0  4.0  5.0
Colorado  6.0  7.0  8.0
--------------------------------------------------
          b     d     e
Utah    0.0   1.0   2.0
Ohio    3.0   4.0   5.0
Texas   6.0   7.0   8.0
Oregon  9.0  10.0  11.0


将这些对象加在一起，返回一个DataFrame，它的索引、列是每个DataFrame的索引、列的并集：

In [75]:
df1 + df2

,b,c,d,e
Colorado,NaN,NaN,NaN,NaN
Ohio,3.0,NaN,6.0,NaN
Oregon,NaN,NaN,NaN,NaN
Texas,9.0,NaN,12.0,NaN
Utah,NaN,NaN,NaN,NaN


由于'c'列和'e'列并不是两个DataFrame共有的列，这两列中产生了缺失值。对于行标签不同的DataFrame对象也是如此。

如果将两个行或列完全不同的DataFrame对象相加，结果将全部为空：

In [76]:
df1 = pd.DataFrame({'A': [1, 2]})
df2 = pd.DataFrame({'B': [3, 4]})

print(df1)
print("-"*50)
print(df2)
print("-"*50)
df1 - df2

   A
0  1
1  2
--------------------------------------------------
   B
0  3
1  4
--------------------------------------------------


,A,B
0,NaN,NaN
1,NaN,NaN


#### 5.2.5.1　使用填充值的算术方法

在两个不同的索引化对象之间进行算术操作时，可能会想要使用特殊填充值，比如当轴标签在一个对象中存在，在另一个对象中不存在时，想将缺失值填充为0：

In [77]:
df1 = pd.DataFrame(np.arange(12.).reshape((3, 4)),
                   columns=list('abcd'))

df2 = pd.DataFrame(np.arange(20.).reshape((4, 5)),
                   columns=list('abcde'))

df2.loc[1, 'b'] = np.nan

df1

,a,b,c,d
0,0.0,1.0,2.0,3.0
1,4.0,5.0,6.0,7.0
2,8.0,9.0,10.0,11.0


In [78]:
df2

,a,b,c,d,e
0,0.0,1.0,2.0,3.0,4.0
1,5.0,NaN,7.0,8.0,9.0
2,10.0,11.0,12.0,13.0,14.0
3,15.0,16.0,17.0,18.0,19.0


将这些df添加到一起会导致在一些不重叠的位置出现NA值：

In [79]:
df1 + df2

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,NaN
1,9.0,NaN,13.0,15.0,NaN
2,18.0,20.0,22.0,24.0,NaN
3,NaN,NaN,NaN,NaN,NaN


在df1上使用add方法，将df2和一个fill_value作为参数传入：

In [80]:
df1.add(df2, fill_value=0)

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,4.0
1,9.0,5.0,13.0,15.0,9.0
2,18.0,20.0,22.0,24.0,14.0
3,15.0,16.0,17.0,18.0,19.0


下表是Series和DataFrame的算术方法。这些方法中的每一个都有一个以r开头的副本，这些副本方法的参数是翻转的。因此下面两个语句的结果是等价的：

| 方法                | 描述       |
| -                   | -         |
| add, radd           | 加法（`+`）    |
| sub, rsub           | 减法（`-`）    |
| div, rdiv           | 除法（`/`）    |
| floordiv, rfloordiv | 整除（`//`）   |
| mul, rmul           | 乘法（`*`）          |
| pow, rpow           | 幂次方（`**`） |

In [81]:
1 / df1

,a,b,c,d
0,inf,1.000000,0.500000,0.333333
1,0.250,0.200000,0.166667,0.142857
2,0.125,0.111111,0.100000,0.090909


In [82]:
df1.div(1)

,a,b,c,d
0,0.0,1.0,2.0,3.0
1,4.0,5.0,6.0,7.0
2,8.0,9.0,10.0,11.0


In [83]:
df1.rdiv(1)

,a,b,c,d
0,inf,1.000000,0.500000,0.333333
1,0.250,0.200000,0.166667,0.142857
2,0.125,0.111111,0.100000,0.090909


与此相关的一点，当对Series或DataFrame重建索引时，也可以指定一个不同的填充值：

In [84]:
df1.reindex(columns=df2.columns, fill_value=0)

,a,b,c,d,e
0,0.0,1.0,2.0,3.0,0
1,4.0,5.0,6.0,7.0,0
2,8.0,9.0,10.0,11.0,0


#### 5.2.5.2　DataFrame和Series间的操作

DataFrame和Series间的算术操作与NumPy中不同维度数组间的操作类似。首先，在下面的示例中，考虑二维数组和其中一行之间的区别：

In [85]:
arr = np.arange(12.).reshape((3, 4))

arr

array([[ 0.,  1.,  2.,  3.],
       [ 4.,  5.,  6.,  7.],
       [ 8.,  9., 10., 11.]])

In [86]:
arr[0]

array([0., 1., 2., 3.])

In [87]:
arr - arr[0]

array([[0., 0., 0., 0.],
       [4., 4., 4., 4.],
       [8., 8., 8., 8.]])

当从arr中减去arr[0]时，减法在每一行都进行了操作。这就是所谓的广播机制，后续将会对涉及通用NumPy数组的内容更深入的解释。DataFrame和Series间的操作是类似的：

In [88]:
frame = pd.DataFrame(np.arange(12.).reshape((4, 3)),
                     columns=list('bde'),
                     index=['Utah', 'Ohio', 'Texas', 'Oregon'])

frame

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [89]:
# 此时是Data Frame
series = frame.iloc[[0], :]
series

,b,d,e
Utah,0.0,1.0,2.0


In [90]:
# 此时是Series
series = frame.iloc[0, :]
series

b    0.0
d    1.0
e    2.0
Name: Utah, dtype: float64

默认情况下，DataFrame和Series的数学操作中会将Series的索引和DataFrame的列进行匹配，并广播到各行：

In [91]:
frame - series

,b,d,e
Utah,0.0,0.0,0.0
Ohio,3.0,3.0,3.0
Texas,6.0,6.0,6.0
Oregon,9.0,9.0,9.0


In [92]:
frame.mul(series)

,b,d,e
Utah,0.0,1.0,4.0
Ohio,0.0,4.0,10.0
Texas,0.0,7.0,16.0
Oregon,0.0,10.0,22.0


如果一个索引值不在DataFrame的列中，也不在Series的索引中，则对象会重建索引并形成联合：

In [93]:
series2 = pd.Series(range(3), index=['b', 'e', 'f'])
series2

b    0
e    1
f    2
dtype: int64

In [94]:
frame + series2

,b,d,e,f
Utah,0.0,NaN,3.0,NaN
Ohio,3.0,NaN,6.0,NaN
Texas,6.0,NaN,9.0,NaN
Oregon,9.0,NaN,12.0,NaN


如果想改为在列上进行广播，在行上匹配，必须使用算术方法中的一种。例如：

In [95]:
series3 = frame['d']

series3

Utah       1.0
Ohio       4.0
Texas      7.0
Oregon    10.0
Name: d, dtype: float64

In [96]:
print(frame.sub(series3, axis='index'))
frame.add(series3, axis=0)

          b    d    e
Utah   -1.0  0.0  1.0
Ohio   -1.0  0.0  1.0
Texas  -1.0  0.0  1.0
Oregon -1.0  0.0  1.0


,b,d,e
Utah,1.0,2.0,3.0
Ohio,7.0,8.0,9.0
Texas,13.0,14.0,15.0
Oregon,19.0,20.0,21.0


传递的axis值是用于匹配轴的。上面的示例中表示需要在DataFrame的行索引上对行匹配（`axis='index'`或`axis=0`），并进行广播。

### 5.2.6　函数应用和映射
NumPy的通用函数（逐元素数组方法）对pandas对象也有效：

In [97]:
frame = pd.DataFrame(np.random.randn(4, 3), columns=list('bde'),
                     index=['Utah', 'Ohio', 'Texas', 'Oregon'])

frame

,b,d,e
Utah,-0.266562,-0.012814,0.145585
Ohio,-2.538733,0.003631,0.749077
Texas,0.244043,1.097734,-0.311277
Oregon,-0.004396,-0.656133,-0.253657


In [98]:
np.abs(frame)

,b,d,e
Utah,0.266562,0.012814,0.145585
Ohio,2.538733,0.003631,0.749077
Texas,0.244043,1.097734,0.311277
Oregon,0.004396,0.656133,0.253657


另一个常用的操作是将函数应用到一行或一列的一维数组上。DataFrame的`apply`方法可以实现这个功能：

In [99]:
f = lambda x: x.max() - x.min()

frame.apply(f)

b    2.782776
d    1.753867
e    1.060355
dtype: float64

这里的函数f，可以计算Series最大值和最小值的差，会被frame中的每一列调用一次。结果是一个以frame的列作为索引的Series。

如果传递`axis='columns'`给apply函数，函数将会被每行调用一次：

In [100]:
frame.apply(f, axis='columns')

Utah      0.412147
Ohio      3.287810
Texas     1.409012
Oregon    0.651737
dtype: float64

大部分最常用的数组统计（比如sum和mean）都是DataFrame的方法，因此计算统计值时使用apply并不是必需的。

传递给apply的函数并不一定要返回一个标量值，也可以返回带有多个值的Series：

In [101]:
def f(x):
    return pd.Series([x.min(), x.max()], index=['min', 'max'])

frame.apply(f)

,b,d,e
min,-2.538733,-0.656133,-0.311277
max,0.244043,1.097734,0.749077


逐元素的Python函数也可以使用。假设你想要根据frame中的每个浮点数计算一个格式化字符串，可以使用applymap方法：

In [102]:
format1 = lambda x: '%.2f' % x

frame.applymap(format1)

,b,d,e
Utah,-0.27,-0.01,0.15
Ohio,-2.54,0.00,0.75
Texas,0.24,1.10,-0.31
Oregon,-0.00,-0.66,-0.25


使用applymap作为函数名是因为Series有map方法，可以将一个逐元素的函数应用到Series上：

In [103]:
frame['e'].map(format1)

Utah       0.15
Ohio       0.75
Texas     -0.31
Oregon    -0.25
Name: e, dtype: object

### 5.2.7　排序和排名
根据某些准则对数据集进行排序是另一个重要的内建操作。如需按行或列索引进行字典型排序，需要使用sort_index方法，该方法返回一个新的、排序好的对象：

In [104]:
obj = pd.Series(range(4), index=['d', 'a', 'c', 'b'])
print(obj)
obj.sort_index()

d    0
a    1
c    2
b    3
dtype: int64


a    1
b    3
c    2
d    0
dtype: int64

在DataFrame中，你可以在各个轴上按索引排序：

In [105]:
frame = pd.DataFrame(np.arange(8).reshape((2, 4)),
                     index=['three', 'one'],
                     columns=['d', 'a', 'b', 'c'])

frame.sort_index()

,d,a,b,c
one,4,5,6,7
three,0,1,2,3


In [106]:
frame.sort_index(axis=1)

,a,b,c,d
three,1,2,3,0
one,5,6,7,4


In [107]:
frame.sort_index().sort_index(axis=1)

,a,b,c,d
one,5,6,7,4
three,1,2,3,0


数据默认会升序排序，但是也可以按照降序排序：

In [108]:
frame.sort_index(axis=1, ascending=False)

,d,c,b,a
three,0,3,2,1
one,4,7,6,5


如果要根据Series的值进行排序，使用sort_values方法：

In [109]:
obj = pd.Series([4, 7, -3, 2])

obj.sort_values()

2   -3
3    2
0    4
1    7
dtype: int64

默认情况下，所有的缺失值都会被排序至Series的尾部：

In [110]:
obj = pd.Series([4, np.nan, 7, np.nan, -3, 2])

obj.sort_values()

4   -3.0
5    2.0
0    4.0
2    7.0
1    NaN
3    NaN
dtype: float64

当对DataFrame排序时，可以使用一列或多列作为排序键。为了实现这个功能，传递一个或多个列名给sort_values的可选参数by：

In [111]:
frame = pd.DataFrame({'b': [4, 7, -3, 2], 'a': [0, 1, 0, 1]})

frame

,b,a
0,4,0
1,7,1
2,-3,0
3,2,1


In [112]:
frame.sort_values(by='b', ascending=False)

,b,a
1,7,1
0,4,0
3,2,1
2,-3,0


对多列排序时，传递列名的列表：

In [113]:
frame.sort_values(by=['a', 'b'])

,b,a
2,-3,0
0,4,0
3,2,1
1,7,1


排名是指对数组从1到有效数据点总数分配名次的操作。Series和DataFrame的rank方法是实现排名的方法，默认情况下，**rank通过将平均排名分配到每个组来打破平级关系**：

In [114]:
obj = pd.Series([7, -5, 7, 4, 2, 0, 4, 7, 0])

obj.rank()

0    8.0
1    1.0
2    8.0
3    5.5
4    4.0
5    2.5
6    5.5
7    8.0
8    2.5
dtype: float64

排名也可以根据他们在数据中的观察顺序进行分配：

In [115]:
obj.rank(method='first')

0    7.0
1    1.0
2    8.0
3    5.0
4    4.0
5    2.0
6    6.0
7    9.0
8    3.0
dtype: float64

在上面的例子中，对条目0和2设置的名次为6和7，而不是之前的平均排名6.5，是因为在数据中标签0在标签2的前面。

你可以按降序排名：

In [116]:
# 将值分配给组中的最大排名
obj.rank(ascending=False, method='max')

0    3.0
1    9.0
2    3.0
3    5.0
4    6.0
5    8.0
6    5.0
7    3.0
8    8.0
dtype: float64

下表是可用的平级关系打破方法列表。

| 方法     | 描述                                                            |
| -       | -                                                               |
| `average` | 默认:在每个组中分配平均排名                                        |
| `min`     | 对整个组使用最小排名                                               |
| `max`     | 对整个组使用最大排名                                               |
| `first`   | 按照值在数据中出现的次序分配排名                                    |
| `dense`   | 类似于 `method='min'`,但组间排名总是增加1,而不是一个组中的相等元素的数量 |

DataFrame可以对行或列计算排名：

In [117]:
frame = pd.DataFrame({'b': [4.3, 7, -3, 2], 'a': [0, 1, 0, 1],
                      'c': [-2, 5, 8, -2.5]})

frame

,b,a,c
0,4.3,0,-2.0
1,7.0,1,5.0
2,-3.0,0,8.0
3,2.0,1,-2.5


In [118]:
frame.rank(axis='columns')

,b,a,c
0,3.0,2.0,1.0
1,3.0,1.0,2.0
2,1.0,2.0,3.0
3,3.0,2.0,1.0


### 5.2.8　含有重复标签的轴索引
目前为止所见过的示例中，轴索引都是唯一的（索引值）。尽管很多pandas函数（比如reindex）需要标签是唯一的，但这个并不是强制性的。考虑一个小型的带有重复索引的Series：

In [119]:
obj = pd.Series(range(5), index=['a', 'a', 'b', 'b', 'c'])
obj

a    0
a    1
b    2
b    3
c    4
dtype: int64

索引的`is_unique`属性可以告诉你它的标签是否唯一：

In [120]:
obj.index.is_unique

False

带有重复索引的情况下，数据选择是与之前操作有差别的主要情况。根据一个标签索引多个条目会返回一个序列，而单个条目会返回标量值：

In [121]:
# 重复索引情况下
print(obj['a'])
print("-"*50)
# unique索引情况下
print(obj['c'])

a    0
a    1
dtype: int64
--------------------------------------------------
4


这可能会使代码更复杂，因为来自索引的输出类型可能因标签是否重复而有所不同。

相同的逻辑可以拓展到在DataFrame中进行行索引：

In [122]:
df = pd.DataFrame(np.random.randn(4, 3), index=['a', 'a', 'b', 'b'])

df

,0,1,2
a,-0.480526,0.549001,-0.403912
a,-0.397779,0.706507,0.507385
b,0.063343,0.715860,0.655230
b,0.250963,0.102219,-0.589159


In [123]:
df.loc['b']

,0,1,2
b,0.063343,0.715860,0.655230
b,0.250963,0.102219,-0.589159


## 5.3　描述性统计的概述与计算
pandas对象装配了一个常用数学、统计学方法的集合。其中大部分属于归约或汇总统计的类别，这些方法从DataFrame的行或列中抽取一个Series或一系列值的单个值（如总和或平均值）。与NumPy数组中的类似方法相比，它们内建了处理缺失值的功能。考虑一个小型DataFrame：

In [124]:
df = pd.DataFrame([[1.4, np.nan], [7.1, -4.5],
                   [np.nan, np.nan], [0.75, -1.3]],
                  index=['a', 'b', 'c', 'd'],
                  columns=['one', 'two'])

df

,one,two
a,1.40,NaN
b,7.10,-4.5
c,NaN,NaN
d,0.75,-1.3


调用DataFrame的sum方法返回一个包含列上加和的Series：

In [125]:
df.sum()

one    9.25
two   -5.80
dtype: float64

传入`axis='columns'`或`axis=1`，则会将一行上各个列的值相加：

In [126]:
df.sum(axis='columns')

a    1.40
b    2.60
c    0.00
d   -0.55
dtype: float64

除非整个切片上（在本例中是行或列）都是NA，否则NA值是被自动排除的。可以通过禁用`skipna`来实现不排除NA值：

In [127]:
df.mean(axis='columns', skipna=False)

a      NaN
b    1.300
c      NaN
d   -0.275
dtype: float64

下表是归约方法的常用可选参数列表。

| 方法   | 描述                                               |
| -      | -                                                  |
| axis   | 归约轴,0为行向,1为列向                               |
| skipna | 排除缺失值,默认为True                               |
| level  | 如果轴是多层索引的( MultiIndex),<br>该参数可以缩减分组层级 |

一些方法，比如`idxmin`和`idxmax`，返回的是间接统计信息，比如最小值或最大值的索引值：

In [128]:
df.idxmax()

one    b
two    d
dtype: object

除了归约方法外，有的方法是积累型方法：

In [129]:
df.cumsum()

,one,two
a,1.40,NaN
b,8.50,-4.5
c,NaN,NaN
d,9.25,-5.8


还有一类方法既不是归约型方法也不是积累型方法。`describe`就是其中之一，它一次性产生多个汇总统计：

In [130]:
df.describe()

,one,two
count,3.000000,2.000000
mean,3.083333,-2.900000
std,3.493685,2.262742
min,0.750000,-4.500000
25%,1.075000,-3.700000
50%,1.400000,-2.900000
75%,4.250000,-2.100000
max,7.100000,-1.300000


对于非数值型数据，`describe`产生另一种汇总统计：

In [131]:
obj = pd.Series(['a', 'a', 'b', 'c'] * 4)

obj.describe()

count     16
unique     3
top        a
freq       8
dtype: object

下表是汇总统计及其相关方法的完整列表。

| 方法           | 描述                                    |
| -              | -                                      |
| count          | 非NA值的个数                            |
| describe       | 计算 Series或 Dataframe各列的汇总统计集合 |
| min, max       | 计算最小值、最大值                        |
| argmin, argmax | 分别计算最小值、最大值所在的索引位置(整数)  |
| idxmin, idxmax | 分别计算最小值或最大值所在的索引标签       |
| quantile       | 计算样本的从0到1间的分位数                |
| sum            | 加和                                    |
| mean           | 均值                                    |
| median         | 中位数(50%分位数)                        |
| mad            | 平均值的平均绝对偏差                      |
| prod           | 所有值的积                              |
| var            | 值的样本方差                             |
| std            | 值的样本标准差                           |
| skew           | 样本偏度(第三时刻)值                     |
| kurt           | 样本峰度(第四时刻)的值                    |
| cumsum         | 累计值                                  |
| cummin, cummax | 累计值的最小值或最大值                    |
| cumprod        | 值的累计积                              |
| diff           | 计算第一个算术差值(对时间序列有用)         |
| pct_change     | 算百分比                                |

### 5.3.1　相关性和协方差
一些汇总统计，比如相关性和协方差，是由多个参数计算出的。考虑某些使用附加pandas-datareader库从Yahoo! Finance上获取的包含股价和交易量的DataFrame。如果还没有安装它，可以通过conda或pip进行安装：
`conda install pandas-datareader`
为了获得一些股票行情，使用pandas_datareader模块下载一些数据：

In [132]:
import pandas_datareader.data as web

In [133]:
all_data = {ticker: web.get_data_yahoo(ticker)
            for ticker in ['AAPL', 'IBM', 'MSFT', 'GOOG']}
price = pd.DataFrame({ticker: data['Adj Close']
                     for ticker, data in all_data.items()})
volume = pd.DataFrame({ticker: data['Volume']
                      for ticker, data in all_data.items()}) 
                      

> 当你看到这里时，Yahoo! Finance可能已经不存在了，因为它在2017年已经被Verizon收购。请参考pandas-datareader的在线文档获取最新功能。但是目前还是可以正常使用。

现在计算股价的百分比，还有一些时间序列操作将会在后面深入探索：

In [134]:
returns = price.pct_change()

In [135]:
returns.tail()

,AAPL,IBM,MSFT,GOOG
Date,,,,
2020-05-11,0.015735,-0.003252,0.011154,0.010725
2020-05-12,-0.011428,-0.019006,-0.022652,-0.019611
2020-05-13,-0.012074,-0.037668,-0.015122,-0.019197
2020-05-14,0.006143,0.010542,0.004339,0.005040
2020-05-15,-0.005912,0.000257,0.014568,0.012580


Series的`corr`方法计算的是两个Series中重叠的、非NA的、按索引对齐的值的相关性。相应地，`cov`计算的是协方差：

In [136]:
returns['MSFT'].corr(returns['IBM'])

0.600353767788083

In [137]:
returns['MSFT'].cov(returns['IBM'])

0.0001621618542275402

由于MSFT是一个有效的Python属性，可以使用更为简洁的语法来获得这些数据：

In [138]:
# 两者的相关系数与顺序无关
returns.IBM.corr(returns.MSFT)

0.600353767788083

另一方面，DataFrame的corr和cov方法会分别以DataFrame的形式返回相关性和协方差矩阵：

In [139]:
returns.corr()

,AAPL,IBM,MSFT,GOOG
AAPL,1.000000,0.532662,0.710548,0.642264
IBM,0.532662,1.000000,0.600354,0.530280
MSFT,0.710548,0.600354,1.000000,0.751001
GOOG,0.642264,0.530280,0.751001,1.000000


In [140]:
returns.cov()

,AAPL,IBM,MSFT,GOOG
AAPL,0.000328,0.000151,0.000221,0.000199
IBM,0.000151,0.000246,0.000162,0.000142
MSFT,0.000221,0.000162,0.000296,0.000221
GOOG,0.000199,0.000142,0.000221,0.000293


使用DataFrame的`corrwith`方法，可以计算出DataFrame中的行或列与另一个序列或DataFrame的相关性。该方法传入一个Series时，会返回一个含有为每列计算相关性值的Series：

In [141]:
returns.corrwith(returns.IBM)

AAPL    0.532662
IBM     1.000000
MSFT    0.600354
GOOG    0.530280
dtype: float64

传入一个DataFrame时，会计算匹配到列名的相关性数值。在这里，计算出交易量与百分比变化的相关性：

In [142]:
returns.corrwith(volume)

AAPL   -0.141855
IBM    -0.105762
MSFT   -0.066296
GOOG   -0.039168
dtype: float64

传入`axis='columns'`会逐行地进行计算。在所有例子中，在计算相关性之前，数据点已经按标签进行了对齐。

### 5.3.2　唯一值、计数和成员属性
另一类相关的方法可以从一维Series包含的数值中提取信息。为了说明这些方法，请考虑这个例子：

In [143]:
obj = pd.Series(['c', 'a', 'd', 'a', 'a', 'b', 'b', 'c', 'c'])

第一个函数是`unique`，它会给出Series中的唯一值：

In [145]:
uniques = obj.unique()

uniques

array(['c', 'a', 'd', 'b'], dtype=object)

唯一值并不一定按照排序好的顺序返回，但是如果需要的话可以进行排序（`uniques.sort()`）。相应地，value_counts计算Series包含的值的个数：

In [146]:
# 统计数量的同时按数量进行排序
obj.value_counts(ascending=True)

d    1
b    2
a    3
c    3
dtype: int64

为了方便，返回的Series会按照数量降序排序。value_counts也是有效的pandas顶层方法，可以用于任意数组或序列：

In [147]:
pd.value_counts(obj.values, sort=False)

d    1
a    3
c    3
b    2
dtype: int64

`isin`执行向量化的成员属性检查，还可以将数据集以Series或DataFrame一列的形式过滤为数据集的值子集：

In [148]:
obj

0    c
1    a
2    d
3    a
4    a
5    b
6    b
7    c
8    c
dtype: object

In [149]:
mask = obj.isin(['b', 'c'])

mask

0     True
1    False
2    False
3    False
4    False
5     True
6     True
7     True
8     True
dtype: bool

In [150]:
obj[mask]

0    c
5    b
6    b
7    c
8    c
dtype: object

与`isin`相关的`Index.get_indexer`方法，可以提供一个索引数组，这个索引数组可以将可能非唯一值数组转换为另一个唯一值数组：

In [151]:
to_match = pd.Series(['c', 'a', 'b', 'b', 'c', 'a'])

unique_vals = pd.Series(['c', 'b', 'a'])

pd.Index(unique_vals).get_indexer(to_match)

array([0, 2, 1, 1, 0, 2])

下表是这些方法的参考。

| 方法         | 描述                                                                  |
| -            | -                                                                    |
| isin         | 计算表征 Series中每个值是否包含于传入序列的布尔值数组                      |
| match        | 计算数组中每个值的整数索引,形成一个唯一值数组。<br>有助于数据对齐和join类型的操作 |
| unique       | 计算 Series值中的唯一值数组,按照观察顺序返回                             |
| value_counts | 返回一个 Series,索引是唯一值序列,值是计数个数,按照个数降序排序             |

某些情况下，可能想要计算DataFrame多个相关列的直方图，如下面的例子：

In [152]:
data = pd.DataFrame({'Qu1': [1, 3, 4, 3, 4],
                     'Qu2': [2, 3, 1, 2, 3],
                     'Qu3': [1, 6, 2, 4, 4]})

data

,Qu1,Qu2,Qu3
0,1,2,1
1,3,3,6
2,4,1,2
3,3,2,4
4,4,3,4


将`pandas.value_counts`传入DataFrame的`apply`函数可以得到：

In [153]:
# 对dataframe中的元素，按行进行value_counts，也就是所谓频次的直方图
result = data.apply(pd.value_counts).fillna(0)

result

,Qu1,Qu2,Qu3
1,1.0,1.0,1.0
2,0.0,2.0,1.0
3,2.0,2.0,0.0
4,2.0,0.0,2.0
6,0.0,0.0,1.0


这里，结果中的行标签是所有列中出现的不同值，数值则是这些不同值在每个列中出现的次数。